In [ ]:
from pyspark.sql import SparkSession

# Create Spark session connected to the cluster
# Using Hadoop 3.3.4 with AWS SDK v1 for better MinIO compatibility
spark = SparkSession.builder \
    .appName("Test Spark Cluster") \
    .master("spark://node-2:7077") \
    .getOrCreate()

# Verify connection
print(f"Spark Version: {spark.version}")
print(f"Master: {spark.sparkContext.master}")
print(f"App Name: {spark.sparkContext.appName}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/21 06:49:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Test basic Spark functionality
# Create a simple DataFrame
data = [
    ("Alice", 34, "Data Engineer"),
    ("Bob", 45, "Data Scientist"),
    ("Charlie", 29, "Developer"),
    ("Diana", 31, "Analyst")
]

df = spark.createDataFrame(data, ["name", "age", "role"])

# Show the DataFrame
print("Original DataFrame:")
df.show()

# Test transformations
print("\nFiltered DataFrame (age > 30):")
df.filter(df.age > 30).show()

# Test aggregations
print("\nAverage age:")
df.agg({"age": "avg"}).show()

# Test groupBy
print("\nCount by role:")
df.groupBy("role").count().show()

print("\n✓ Spark cluster is working correctly!")


Original DataFrame:


+-------+---+--------------+
|   name|age|          role|
+-------+---+--------------+
|  Alice| 34| Data Engineer|
|    Bob| 45|Data Scientist|
|Charlie| 29|     Developer|
|  Diana| 31|       Analyst|
+-------+---+--------------+


Filtered DataFrame (age > 30):
+-----+---+--------------+
| name|age|          role|
+-----+---+--------------+
|Alice| 34| Data Engineer|
|  Bob| 45|Data Scientist|
|Diana| 31|       Analyst|
+-----+---+--------------+


Average age:


+--------+
|avg(age)|
+--------+
|   34.75|
+--------+


Count by role:


+--------------+-----+
|          role|count|
+--------------+-----+
|Data Scientist|    1|
| Data Engineer|    1|
|     Developer|    1|
|       Analyst|    1|
+--------------+-----+


✓ Spark cluster is working correctly!


In [ ]:
# Write file to min io
print("=== Writing to Bronze Layer ===")
bronze_path = "s3a://delta-lake/bronze/orders"

sample_data = [
    {"order_id": 1, "customer_id": 101, "amount": 250.50, "status": "completed"},
    {"order_id": 2, "customer_id": 102, "amount": 175.00, "status": "pending"},
    {"order_id": 3, "customer_id": 103, "amount": 320.75, "status": "completed"},
]

df_bronze = spark.createDataFrame(sample_data)
df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .save(bronze_path)

print(f"✓ Bronze data written to: {bronze_path}")